#Introdução

Este notebook contém a parte prática do trabalho de conclusão de curso titulado "Aprendizado de máquina para análise de tráfego de rede e detecção de intrusão".

In [ ]:
!pip install sweetviz mljar-supervised ipywidgets

In [1]:
import pandas as pd
import numpy as np
import sklearn
import sweetviz as sv
from supervised.automl import AutoML # mljar-supervised
from sklearn.metrics import accuracy_score
from ipywidgets import IntProgress

In [3]:
path_Train = "./KDDTrain+.txt"
path_Test = "./KDDTest+.txt"

#Leitura e adição de Headers 

In [4]:
df_train= pd.read_csv(path_Train,header=None)
df_test= pd.read_csv(path_Test,header=None)

In [6]:
columns = (['duration'
,'protocol_type'
,'service'
,'flag'
,'src_bytes'
,'dst_bytes'
,'land'
,'wrong_fragment'
,'urgent'
,'hot'
,'num_failed_logins'
,'logged_in'
,'num_compromised'
,'root_shell'
,'su_attempted'
,'num_root'
,'num_file_creations'
,'num_shells'
,'num_access_files'
,'num_outbound_cmds'
,'is_host_login'
,'is_guest_login'
,'count'
,'srv_count'
,'serror_rate'
,'srv_serror_rate'
,'rerror_rate'
,'srv_rerror_rate'
,'same_srv_rate'
,'diff_srv_rate'
,'srv_diff_host_rate'
,'dst_host_count'
,'dst_host_srv_count'
,'dst_host_same_srv_rate'
,'dst_host_diff_srv_rate'
,'dst_host_same_src_port_rate'
,'dst_host_srv_diff_host_rate'
,'dst_host_serror_rate'
,'dst_host_srv_serror_rate'
,'dst_host_rerror_rate'
,'dst_host_srv_rerror_rate'
,'attack'
,'level'])

df_train.columns = columns
df_test.columns = columns

#Tratamento da variável categórica de ataque
Nesta parte, a variável categórica de ataque é transformada em 0 ou 1 indicando a presença de um ataque.
* 0 simboliza tráfego normal
* 1 simboliza tráfego inseguro

In [5]:
is_attack = df_train.attack.map(lambda a: 0 if a == 'normal' else 1)
test_attack = df_test.attack.map(lambda a: 0 if a == 'normal' else 1)

df_train['attack_flag'] = is_attack
df_test['attack_flag'] = test_attack

df_train.head()

AttributeError: 'DataFrame' object has no attribute 'attack'

#Seleção de colunas relevantes
Durante esta etapa foram selecionadas as colunas que contém informações relevantes para a determinação dos ataques.

As colunas significativas são:

same_srv_rate, dst_host_srv_count, dst_host_same_srv_rate,dst_host_srv_serror_rate, dst_host_serror_rate, serror_rate,dst_host_diff_srv_rate, diff_srv_rate, srv_diff_host_rate, service, flag,logged_in, srv_serror_rate, count, level, dst_host_count, srv_rerror_rate, dst_host_srv_rerror_rate, rerror_rate, dst_host_rerror_rate, attack_flag

In [8]:
df_del = df_train.drop(['same_srv_rate','dst_host_srv_count','dst_host_same_srv_rate','dst_host_srv_serror_rate','service','flag','logged_in','dst_host_serror_rate','serror_rate','dst_host_diff_srv_rate','diff_srv_rate','srv_diff_host_rate','srv_serror_rate','count','level','dst_host_count','srv_rerror_rate','dst_host_srv_rerror_rate','rerror_rate','dst_host_rerror_rate'],axis=1)

In [2]:
cor_ananlysis = sv.analyze(df_train,target_feat='attack_flag')

NameError: name 'df_train' is not defined

In [ ]:
cor_ananlysis.show_notebook()

Como é possível ver, a tabela de correlações está bem mais adequada à coluna de 'attack_flag'

In [ ]:
df_test_new = df_test[['same_srv_rate','dst_host_srv_count','dst_host_same_srv_rate','dst_host_srv_serror_rate','dst_host_serror_rate','serror_rate','dst_host_diff_srv_rate','diff_srv_rate','srv_diff_host_rate','service','flag','logged_in','srv_serror_rate','count','level','dst_host_count','srv_rerror_rate','dst_host_srv_rerror_rate','rerror_rate','dst_host_rerror_rate','attack_flag']]
df_train_new = df_train[['same_srv_rate','dst_host_srv_count','dst_host_same_srv_rate','dst_host_srv_serror_rate','dst_host_serror_rate','serror_rate','dst_host_diff_srv_rate','diff_srv_rate','srv_diff_host_rate','service','flag','logged_in','srv_serror_rate','count','level','dst_host_count','srv_rerror_rate','dst_host_srv_rerror_rate','rerror_rate','dst_host_rerror_rate','attack_flag']]

In [ ]:
cor_ananlysis2 = sv.analyze(df_train_new,target_feat='attack_flag')

In [ ]:
cor_ananlysis2.show_notebook()

In [ ]:
X_train,Y_train = df_train_new.drop(['attack_flag'],axis=1),df_train_new['attack_flag']
X_test,Y_test = df_test_new.drop(['attack_flag'],axis=1),df_test_new['attack_flag']

In [ ]:
automl = AutoML(results_path="AutoML_classifier",mode="Perform")
automl.fit(X_train,Y_train)

In [ ]:
prediction = automl.predict(X_test)

In [ ]:
accuracy_score(Y_test,prediction)